In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
!kaggle datasets download abdulhasibuddin/plant-doc-dataset


Dataset URL: https://www.kaggle.com/datasets/abdulhasibuddin/plant-doc-dataset
License(s): unknown
100% 881M/882M [00:25<00:00, 36.6MB/s]
100% 882M/882M [00:25<00:00, 36.7MB/s]


In [ ]:
!unzip plant-doc-dataset.zip -d plantdoc

Archive:  plant-doc-dataset.zip
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (1).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (10).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (2).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (3).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (4).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (5).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (6).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (7).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (8).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple Scab Leaf/Apple Scab Leaf (9).jpg  
  inflating: plantdoc/PlantDoc-Dataset/test/Apple leaf/Apple leaf (1).jpg  
  inflating: plantdoc/PlantDoc-

In [9]:
# Define dataset paths
data_dir = "/content/plantdoc/PlantDoc-Dataset"  # Change this to the correct path
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")


In [10]:
# Image parameters
img_height, img_width = 224, 224  # Resize images
batch_size = 32

# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1866 images belonging to 27 classes.
Found 450 images belonging to 27 classes.


In [11]:
# Define CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')  # Changed to softmax for multi-class classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(train_data, validation_data=val_data, epochs=20)

# Evaluate model
test_data = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Changed to categorical for multi-class classification
)

Epoch 1/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 41s 635ms/step - accuracy: 0.0488 - loss: 3.4813 - val_accuracy: 0.0889 - val_loss: 3.2385
Epoch 2/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 35s 592ms/step - accuracy: 0.0772 - loss: 3.2545 - val_accuracy: 0.0911 - val_loss: 3.2136
Epoch 3/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 35s 592ms/step - accuracy: 0.0907 - loss: 3.2000 - val_accuracy: 0.1200 - val_loss: 3.1312
Epoch 4/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 35s 589ms/step - accuracy: 0.1091 - loss: 3.0827 - val_accuracy: 0.1622 - val_loss: 3.0408
Epoch 5/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 45s 655ms/step - accuracy: 0.1484 - loss: 2.9457 - val_accuracy: 0.1156 - val_loss: 3.0353
Epoch 6/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 34s 584ms/step - accuracy: 0.2136 - loss: 2.6816 - val_accuracy: 0.1244 - val_loss: 3.0485
Epoch 7/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 35s 590ms/step - accuracy: 0.3197 - loss: 2.2778 - val_accuracy: 0.1622 - val_loss: 3.1128
Epoch 8/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 33s 564ms/step - accuracy: 0.4576 - loss: 1.8759 - val_accu

In [14]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy:.4f}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 397ms/step - accuracy: 0.1490 - loss: 6.0847
Test Accuracy: 0.1356


In [15]:
model.save("plantdoc_model.h5")